In [21]:
from ProbRobNLP.formal_parser import *
from ProbRobNLP.probrob import *
from collections import defaultdict

In [2]:
f = """
from model import *

table = Table on Vector3D(0,0,0), with width 1.8, with length 0.8, with height 0.8
r1 = Robot on (top back table).position - Vector3D(0.4, 0, 0), with color "0.5"
r2 = Robot on (top back table).position + Vector3D(0.4, 0, 0)

tr_1 = Tray completely on table, ahead of r1 by 0.1, left of (top table) by 0.2
tr_2 = Tray completely on table, ahead of r2 by 0.1, right of (top table) by 0.2

c1 = Cube completely on tr_1

camera = Camera at Vector3D(table.x + (-0.1, 0.1), table.y + (-0.1, 0.1), (1.9, 2.1)), facing Vector3D(0, 0, -1)
"""
lexer = PRSLexer()
parser = PRSParser()
result = parser.parse(lexer.tokenize(f))
result

from model import *
table = Table on Vector3D(0.0, 0.0, 0.0), with width 1.8, with length 0.8, with height 0.8
r1 = Robot on (top back table).position - Vector3D(0.4, 0.0, 0.0), with color 0.5
r2 = Robot on (top back table).position + Vector3D(0.4, 0.0, 0.0)
tr_1 = Tray completely on table, ahead of r1 by 0.1, left of (top table) by 0.2
tr_2 = Tray completely on table, ahead of r2 by 0.1, right of (top table) by 0.2
c1 = Cube completely on tr_1
camera = Camera at Vector3D(table.x + (-0.1, 0.1), table.y + (-0.1, 0.1), (1.9, 2.1)), facing Vector3D(0.0, 0.0, -1.0)

In [19]:
for entity in result.entities:
    print(entity.name, entity.constraints, entity.type)

table [on Vector3D(0.0, 0.0, 0.0), with width 1.8, with length 0.8, with height 0.8] Table
r1 [on (top back table).position - Vector3D(0.4, 0.0, 0.0), with color 0.5] Robot
r2 [on (top back table).position + Vector3D(0.4, 0.0, 0.0)] Robot
tr_1 [completely on table, ahead of r1 by 0.1, left of (top table) by 0.2] Tray
tr_2 [completely on table, ahead of r2 by 0.1, right of (top table) by 0.2] Tray
c1 [completely on tr_1] Cube
camera [at Vector3D(table.x + (-0.1, 0.1), table.y + (-0.1, 0.1), (1.9, 2.1)), facing Vector3D(0.0, 0.0, -1.0)] Camera


In [54]:
class State():
    
    def __init__(self):
        self.objects = set()
        self.constraints = defaultdict(list)
        self.types = {}
        self.imports = []

    def from_world_state(worldmodel):
        self = State()
        self.imports = worldmodel.imports
        for entity in worldmodel.entities:
            self.objects.add(entity.name)
            self.types[entity.name] = entity.type
            for constraint in entity.constraints:
                self.constraints[entity.name].append(constraint)
        return self
    

    def add_constraint(self, o, constraint):
        if o in self.objects:
            self.constraints[o].append(constraint)
        else:
            raise ValueError(f"attempted to add constraint to object which does not exist: {o}")
    
    def add_object(self, o, t):
        self.objects.add(o)
        self.types[o] = t
        
    def to_world_state(self):
        entities = [Entity(self.types[o], o, self.constraints[o]) for o in self.objects]
        return WorldModel(entities, self.imports)
    
    
    def abstract_add_object(self, o, t):
        
        def add_object():
            self.objects.add(o)
            self.types[o] = t
        return add_object
    
    def abstract_add_constraint(self, o, constraint):
        
        def add_constraint():
            if o in self.objects:
                self.constraints[o].append(constraint)
            else:
                raise ValueError(f"attempted to add constraint to object which does not exist: {o}")
        return add_constraint
    
    def __repr__(self):
        ws = self.to_world_state()
        return str(ws)

In [28]:
d = State.from_world_state(result)
d.to_world_state()

from model import *
camera = Camera at Vector3D(table.x + (-0.1, 0.1), table.y + (-0.1, 0.1), (1.9, 2.1)), facing Vector3D(0.0, 0.0, -1.0)
r2 = Robot on (top back table).position + Vector3D(0.4, 0.0, 0.0)
r1 = Robot on (top back table).position - Vector3D(0.4, 0.0, 0.0), with color 0.5
tr_2 = Tray completely on table, ahead of r2 by 0.1, right of (top table) by 0.2
table = Table on Vector3D(0.0, 0.0, 0.0), with width 1.8, with length 0.8, with height 0.8
c1 = Cube completely on tr_1
tr_1 = Tray completely on table, ahead of r1 by 0.1, left of (top table) by 0.2

In [16]:
d.constraints

defaultdict(list,
            {'table': [on Vector3D(0.0, 0.0, 0.0),
              with width 1.8,
              with length 0.8,
              with height 0.8],
             'r1': [on (top back table).position - Vector3D(0.4, 0.0, 0.0),
              with color 0.5],
             'r2': [on (top back table).position + Vector3D(0.4, 0.0, 0.0)],
             'tr_1': [completely on table,
              ahead of r1 by 0.1,
              left of (top table) by 0.2],
             'tr_2': [completely on table,
              ahead of r2 by 0.1,
              right of (top table) by 0.2],
             'c1': [completely on tr_1],
             'camera': [at Vector3D(table.x + (-0.1, 0.1), table.y + (-0.1, 0.1), (1.9, 2.1)),
              facing Vector3D(0.0, 0.0, -1.0)]})

In [63]:
class SemanticOptions:
    
    def __init__(self, l):
        self.list_of_options = l
    
    def evaluate_option(self, i):
        for f in self.list_of_options[i]:
            f()
    
    def __repr__(self):
        out = []
        for i, option in enumerate(self.list_of_options):
            out.append(f"Option {i}: " + ', '.join([str(o) for o in option]))
        return '\n'.join(out)

In [71]:
d = State()
imports = [Import('model', '*')]
d.imports = imports

"there is a table at the centre"
centre = Vector3D(0, 0, 0)
s1 = SemanticOptions([[d.abstract_add_object('table', 'Table'), d.abstract_add_constraint('table', AtConstraint(centre))], 
 [d.abstract_add_object('table', 'Table'), d.abstract_add_constraint('table', OnConstraint(centre))]])
s1.evaluate_option(1)


"the table should be 1.8 wide, 0.8 long, and 0.8 high"
s2 = SemanticOptions([[d.abstract_add_constraint('table', PropConstraint('width', 1.8)),
                      d.abstract_add_constraint('table', PropConstraint('length', 0.8)),
                      d.abstract_add_constraint('table', PropConstraint('height', 0.8))]])

s2.evaluate_option(0)


m = Minus(PropReference(SideSpec('table', Side(['top', 'back'])), 'position'), Vector3D(.4, 0, 0))
"there is a gray robot 0.4 to the left of the back of the table"
s3 = SemanticOptions([[d.abstract_add_object('r1', 'Robot'), 
                       d.abstract_add_constraint('r1', OnConstraint(m))],
                       d.abstract_add_constraint('r1', PropConstraint('color', '"0.5"'))])
          
s3.evaluate_option(0)

p = Plus(PropReference(SideSpec('table', Side(['top', 'back'])), 'position'), Vector3D(.4, 0, 0))
"there is a robot 0.4 to the right of the back of the table"
s4 = SemanticOptions([[d.abstract_add_object('r2', 'Robot'), 
                       d.abstract_add_constraint('r2', OnConstraint(p))],
                       d.abstract_add_constraint('r2', PropConstraint('color', '"0.5"'))])
          
s4.evaluate_option(0)
d

from model import *
r2 = Robot on (top back table).position + Vector3D(0.4, 0, 0)
table = Table on Vector3D(0, 0, 0), with width 1.8, with length 0.8, with height 0.8
r1 = Robot on (top back table).position - Vector3D(0.4, 0, 0)

In [57]:
f = """
from model import *

table = Table on Vector3D(0,0,0), with width 1.8, with length 0.8, with height 0.8
r1 = Robot on (top back table).position - Vector3D(0.4, 0, 0), with color "0.5"
r2 = Robot on (top back table).position + Vector3D(0.4, 0, 0)

tr_1 = Tray completely on table, ahead of r1 by 0.1, left of (top table) by 0.2
tr_2 = Tray completely on table, ahead of r2 by 0.1, right of (top table) by 0.2

c1 = Cube completely on tr_1

camera = Camera at Vector3D(table.x + (-0.1, 0.1), table.y + (-0.1, 0.1), (1.9, 2.1)), facing Vector3D(0, 0, -1)
"""

In [59]:
"there is a table at the centre"
centre = Vector3D(0, 0, 0)
s1 = [[d.abstract_add_object('table', 'Table'), d.abstract_add_constraint('table', AtConstraint(centre))], 
 [d.abstract_add_object('table', 'Table'), d.abstract_add_constraint('table', OnConstraint(centre))]]

In [61]:
SemanticOptions(s1).evaluate_option(0)